# Isolation

This IPython notebook contains the skeletons of the player class and eval function classes that you need to fill out. In addition, we have included the `RandomPlayer` and `HumanPlayer` classes for you to test against.

## Submitting

When you are ready to submit, copy code from the following classes into the attached `player_submission.py`:

1. OpenMoveEvalFn
1. CustomEvalFn
1. CustomPlayer

Please do not copy any code that is not part of those classes. You may be tempted to simply export a python file from this notebook and use that as a submission; please **DO NOT** do that. We need to be certain that code unprotected by a *main* test (any tests that you might want to write) do not get accidentally executed

## Helper Player classes

We include 2 player types for you to test against locally:

- `RandomPlayer` - chooses a legal move randomly from among the available legal moves
- `HumanPlayer` - allows *YOU* to play against the AI

**DO NOT** submit.

You are however free to change these classes as you see fit. Know that any changes you make will be solely for the benefit of your own tests.

In [4]:
from random import randint

class RandomPlayer():
    """Player that chooses a move randomly."""
    def move(self, game, legal_moves, time_left):
        if not legal_moves: return (-1,-1)
        return legal_moves[randint(0,len(legal_moves)-1)]

In [5]:
class HumanPlayer():
    """Player that chooses a move according to
    user's input."""
    def move(self, game, legal_moves, time_left):
        print('\t'.join(['[%d] %s'%(i,str(move)) for i,move in enumerate(legal_moves)] ))
        
        valid_choice = False
        while not valid_choice:
            try:
                index = int(raw_input('Select move index:'))
                valid_choice = 0 <= index < len(legal_moves)

                if not valid_choice:
                    print('Illegal move! Try again.')
            
            except ValueError:
                print('Invalid index! Try again.')
        return legal_moves[index]

## Evaluation Functions

These functions will inform the value judgements your AI will make when choosing moves. There are 2 classes:

- `OpenMoveEvalFn` - Scores the number of moves open for your player. All baseline tests will use this function. Mandatory
- `CustomEvalFn` - You are encouraged to create your own evaluation function here.

**DO** submit code within the classes (and only that within the classes).

### Tips

1. You may write additional code within each class. However, we will only be invoking the `score()` function. You may not change the signature of this function.
1. When writing additional code to test, try to do so in separate cells. It allows for independent test execution and you can be sure that *all* the code within the EvalFn cells belong only to the EvalFn classes

In [6]:
class OpenMoveEvalFn():
    """Evaluation function that outputs a 
    score equal to how many moves are open
    for your computer player on the board."""
    def score(self, game, maximizing_player_turn=True):
        # TODO: finish this function!
        return len(game.get_legal_moves())

In [7]:
class CustomEvalFn():
    """Custom evaluation function that acts
    however you think it should. This is not
    required but highly encouraged if you
    want to build the best AI possible."""
    def score(self, game, maximizing_player_turn=True):
        # TODO: finish this function!
        return len(game.get_legal_moves()) - len(game.get_opponent_moves())

### Tests

We've included some sample code to test the evaluation functions. Change as you see fit. **DO NOT** submit.

In [8]:
"""Example test you can run
to make sure your basic evaluation
function works."""
from isolation import Board

if __name__ == "__main__":
    sample_board = Board(RandomPlayer(),RandomPlayer())
    # setting up the board as though we've been playing
    sample_board.move_count = 3
    sample_board.__active_player__ = 0 # player 1 = 0, player 2 = 1
    # 1st board = 7 moves
    sample_board.__board_state__ = [
                [0,2,0,0,0,0,0],
                [0,0,0,0,0,0,0],
                [0,0,1,0,0,0,0],
                [0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0]
    ]
    sample_board.__last_player_move__ = {0: (2,2), 1: (0,1)}

    # player 1 should have 7 moves available,
    # so board gets a score of 7
    h = OpenMoveEvalFn()
    print('This board has a score of %s.'%(h.score(sample_board)))

This board has a score of 7.


## CustomPlayer

This is the meat of the assignment. A few notes about the class:

- You are not permitted to change the function signatures of any of the provided methods.
- You are permitted to change the default values within the function signatures provided. In fact, when you have your custom evaluation function, you are encouraged to change the default values for `__init__` to use the new eval function.
- You are free change the contents of each of the provided methods. When you are ready with `alphabeta()`, for example, you are encouraged to update `move()` to use that function instead.
- You are free to add more methods to the class.
- You may not create additional external functions and classes that are referenced from within this class.

**DO** submit the code within this class (and only the code within this class).

In [9]:
class CustomPlayer():
    # TODO: finish this class!
    """Player that chooses a move using
    your evaluation function and
    a depth-limited minimax algorithm
    with alpha-beta pruning.
    You must finish and test this player
    to make sure it properly uses minimax
    and alpha-beta to return a good move
    in less than 1000 milliseconds."""
    def __init__(self, search_depth=3, eval_fn=CustomEvalFn()):
        # if you find yourself with a superior eval function, update the
        # default value of `eval_fn` to `CustomEvalFn()`
        self.eval_fn = eval_fn
        self.search_depth = search_depth
        self.opening_moves = [(3,3), (0,3), (3,6), (6,3), (3,0)]

    def move(self, game, legal_moves, time_left):
        if len(legal_moves) == 1:
            return legal_moves[0]
        if game.move_count < 2:
            for move in self.opening_moves:
                if game.move_is_legal(move[0], move[1]):
                    return move
        else:
            # Attempt to reflect opponents move
            if game.move_count >= 4:
                best_move = self.get_reflected_move(game, legal_moves)
                if game.move_is_legal(best_move[0], best_move[1]):
                    return best_move
                
            #best_move, utility = self.minimax(game, depth=self.search_depth)
            #return best_move
            # you will eventually replace minimax with alpha-beta

            # Alpha-beta pruning
            #best_move, utility = self.alphabeta(game, depth=self.search_depth)
            #return best_move

            # Iterative Deepening
            self.search_depth = len(game.get_blank_spaces())
            best_moves = {}
            id_level = 1
            #print "Move %d ####################" % game.move_count
            #print "Search Depth: %d" % self.search_depth
            while time_left() > 250 and id_level <= self.search_depth:
                #print "Time left:  %d" % time_left()
                #print "Iterative Deepening Level: %d" % id_level
                # Alpha-beta pruning
                best_move, utility = self.alphabeta(game, time_left, depth=self.search_depth)
                #print "Time left:  %d" % time_left()
                best_moves[utility] = best_move
                id_level += 1
            #print "Eval Result: %f" % max(best_moves.keys(), key=float)
            #print "############################"
            best_move = best_moves[max(best_moves.keys(), key=float)]
            return best_move

    def utility(self, game, maximizing_player=True):
        """TODO: Update this function to calculate the utility of a game state"""

        if game.is_winner(self):
            return float("inf")

        if game.is_opponent_winner(self):
            return float("-inf")

        return self.eval_fn.score(game, maximizing_player)

    def minimax(self, game, time_left, depth=float("inf"), maximizing_player=True):
        # TODO: finish this function!
        if self.cutoff_test(game, time_left, depth):
            return None, self.utility(game)
        results = []
        actions = game.get_legal_moves()
        for action in actions:
            if maximizing_player:
                results.append(self.min_value(game.forecast_move(action), time_left, depth - 1, not maximizing_player))
            else:
                results.append(self.max_value(game.forecast_move(action), time_left, depth - 1, not maximizing_player))
        best_val = max(results) if maximizing_player else min(results)
        best_index = results.index(best_val)
        best_move = actions[best_index]
        return best_move, best_val

    def max_value(self, game, time_left, depth, maximizing_player):
        if self.cutoff_test(game, time_left, depth):
            return self.utility(game, maximizing_player)
        v = float("-inf")
        results = []
        actions = game.get_legal_moves()
        for action in actions:
            results.append(self.min_value(game.forecast_move(action), time_left, depth - 1, not maximizing_player))
        if len(results) > 0:
            v = max(results)
        return v
    
    def min_value(self, game, time_left, depth, maximizing_player):
        if self.cutoff_test(game, time_left, depth):
            return self.utility(game, maximizing_player)
        v = float("inf")
        results = []
        actions = game.get_legal_moves()
        for action in actions:
            results.append(self.max_value(game.forecast_move(action), time_left, depth - 1, not maximizing_player))
        if len(results) > 0:
            v = min(results)
        return v

    def cutoff_test(self, game, time_left, depth):
        #print "CUTOFF: Depth: %d" % depth
        #print "CUTOFF: Time left: %d" % time_left()
        return depth == 0 or len(game.get_legal_moves()) == 0 or time_left() < 60

    def alphabeta(self, game, time_left, depth=float("inf"), alpha=float("-inf"), beta=float("inf"), maximizing_player=True):
        # TODO: finish this function!
        if self.cutoff_test(game, time_left, depth):
            return None, self.utility(game)
        results = []
        actions = game.get_legal_moves()
        for action in actions:
            if maximizing_player:
                results.append(self.min_value_AB(game.forecast_move(action),
                                                 time_left,
                                                 depth - 1, 
                                                 not maximizing_player,
                                                 alpha,
                                                 beta))
            else:
                results.append(self.max_value_AB(game.forecast_move(action), 
                                                 time_left,
                                                 depth - 1, 
                                                 not maximizing_player,
                                                 alpha, 
                                                 beta))
        best_val = max(results) if maximizing_player else min(results)
        best_index = results.index(best_val)
        best_move = actions[best_index]
        return best_move, best_val
    
    def max_value_AB(self, game, time_left, depth, maximizing_player, alpha, beta):
        if self.cutoff_test(game, time_left, depth):
            return self.utility(game, maximizing_player)
        v = float("-inf")
        results = []
        actions = game.get_legal_moves()
        for action in actions:
            results.append(self.min_value_AB(game.forecast_move(action), 
                                             time_left,
                                             depth - 1, 
                                             not maximizing_player,
                                             alpha,
                                             beta))
            if len(results) > 0:
                v = max(results)
            if v >= beta:
                return v
            alpha = max([alpha, v])
        return v
    
    def min_value_AB(self, game, time_left, depth, maximizing_player, alpha, beta):
        if self.cutoff_test(game, time_left, depth):
            return self.utility(game, maximizing_player)
        v = float("inf")
        results = []
        actions = game.get_legal_moves()
        for action in actions:
            results.append(self.max_value_AB(game.forecast_move(action), 
                                             time_left,
                                             depth - 1, 
                                             not maximizing_player,
                                             alpha, 
                                             beta))
            if len(results) > 0:
                v = min(results)
            if v <= alpha:
                return v
            beta = min([beta, v])
        return v

    def get_reflected_move(self, game, legal_moves):
        opponent = game.get_inactive_player()
        opponent_locations = game.get_player_locations(opponent)
        opponent_quadrant = self.get_quadrant(opponent_locations[len(opponent_locations) - 2],
                                         opponent_locations[len(opponent_locations) - 1])
        last_location = game.get_last_move_for_player(self)
        reflected_moves = {}
        for move in legal_moves:
            utility = float("-inf")
            quadrant = self.get_quadrant(last_location, move)
            if opponent_quadrant == 0 and quadrant == 2:
                utility = self.eval_fn.score(game.forecast_move(move))
                reflected_moves[utility] = move
            if opponent_quadrant == 1 and quadrant == 3:
                utility = self.eval_fn.score(game.forecast_move(move))
                reflected_moves[utility] = move
            if opponent_quadrant == 2 and quadrant == 0:
                utility = self.eval_fn.score(game.forecast_move(move))
                reflected_moves[utility] = move
            if opponent_quadrant == 3 and quadrant == 1:
                utility = self.eval_fn.score(game.forecast_move(move))
                reflected_moves[utility] = move
        if len(reflected_moves.keys()) == 0:
            return (-1, -1)
        return reflected_moves[max(reflected_moves.keys(), key=float)]

    def get_quadrant(self, a, b):
        dx = b[0] - a[0]
        dy = b[1] - a[1]
        if dx < 0 and dy < 0:
            return 0
        if dx < 0 and dy > 0:
            return 1
        if dx > 0 and dy > 0:
            return 2
        if dx < 0 and dy > 0:
            return 3

### Tests

We've included some code to help you test your player as well as to give you an idea of how the players are invoked. Feel free to play around with the code and add more tests.

**DO NOT** submit.

In [10]:
"""Example test to make sure
your minimax works, using the
#my_moves evaluation function."""
from isolation import Board, game_as_text

if __name__ == "__main__":
    # create dummy 3x3 board
    p1 = CustomPlayer(search_depth=3)
    p2 = CustomPlayer()
    b = Board(p1,p2,3,3)
    b.__board_state__ = [
        [0,2,0],
        [0,0,1],
        [0,0,0]
    ]
    b.__last_player_move__[p1] = (1,2)
    b.__last_player_move__[p2] = (0,1)
    b.move_count = 2
    
    output_b = b.copy()
    # use minimax to determine optimal move 
    # sequence for player 1
    winner, move_history, termination = b.play_isolation(time_limit=500)
    
    print game_as_text(winner, move_history, termination, output_b)
    # your output should look like this:
    """
    ####################
      | 2 |   | 
      |   | 1 | 
      |   |   | 

    ####################
    ####################
    1 | 2 |   | 
      |   | - | 
      |   |   | 

    ####################
    ####################
    1 | - |   | 
      |   | - | 
      |   | 2 | 

    ####################
    ####################
    - | - |   | 
      |   | - | 
      | 1 | 2 | 

    ####################
    ####################
    - | - |   | 
    2 |   | - | 
      | 1 | - | 

    ####################
    ####################
    - | - | 1 | 
    2 |   | - | 
      | - | - | 

    ####################
    Illegal move at -1,-1.
    Player 1 wins.
    """

-1. Initial board state
  | 2 |   | 
  |   | 1 | 
  |   |   | 
0. (0,0)
1 | 2 |   | 
  |   | - | 
  |   |   | 
0. ... (2,0)
1 | - |   | 
  |   | - | 
2 |   |   | 
1. (2,1)
- | - |   | 
  |   | - | 
2 | 1 |   | 
1. ... (-1,-1)
- | - |   | 
  |   | - | 
2 | 1 |   | 
illegal move at (-1,-1)
Winner: Player 1



In [11]:
"""Example test you can run
to make sure your AI does better
than random."""
from isolation import Board, game_as_text

if __name__ == "__main__":
    r = RandomPlayer()
    h = CustomPlayer()
    game = Board(h,r)
    output = game.copy()
    winner, move_history, termination = game.play_isolation(time_limit=500)
    print game_as_text(winner, move_history, termination, output)

-1. Initial board state
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 
0. (3,3)
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 
  |   |   | 1 |   |   |   | 
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 
0. ... (4,6)
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 
  |   |   | 1 |   |   |   | 
  |   |   |   |   |   | 2 | 
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 
1. (1,4)
  |   |   |   |   |   |   | 
  |   |   |   | 1 |   |   | 
  |   |   |   |   |   |   | 
  |   |   | - |   |   |   | 
  |   |   |   |   |   | 2 | 
  |   |   |   |   |   |   | 
  |   |   |   |   |   |   | 
1. ... (6,5)
  |   |   |   |   |   |   | 
  |   |   |   | 1 |   |   | 
  |   |   |   |   |   |   | 
  |   |   | - |   |   |   | 
  | 